In [1]:
%pip install captcha 
%pip install opencv-python
!apt update && apt install -y libsm6 libxext6
!pip install opencv-python-headless
!pip install opencv-contrib-python-headless
%pip install keras


import argparse
import json
import string
import os
import shutil
import uuid
from captcha.image import ImageCaptcha

import itertools

import os
import cv2
import numpy as np
from random import random, randint, choices

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input
import matplotlib.pyplot as plt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


'apt' is not recognized as an internal or external command,
operable program or batch file.


Note: you may need to restart the kernel to use updated packages.


In [3]:
BATCH_SIZE = 128
EPOCHS = 50
IMG_ROW, IMG_COLS = 50, 150


# Non-configs
PATH = os.getcwd()
DATA_PATH = os.path.join(PATH, 'train\\gen_bad_captcha')

In [13]:
def load_data(path, test_split=0.1, width=IMG_COLS, height=IMG_ROW):
    print ('loading dataset...')
    y_train = []
    y_test = []
    x_train = []
    x_test = []

    # r=root, d=directories, f = files
    counter = 0
    for r, d, f in os.walk(path):
        for fl in f:
            if '.png' in fl:
                flr = fl.split('_')[0]
                counter += 1
                if(flr=='bad'):
                    label = np.array([0,1])
                else:
                    label = np.array([1,0])

                img = cv2.imread(os.path.join(r, fl),cv2.IMREAD_UNCHANGED)
                img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                img = cv2.resize(img, (int(width/2), int(height/2)), interpolation=cv2.INTER_AREA)
                img = np.reshape(img, (img.shape[0], img.shape[1], 1))

                if random() < test_split:
                    y_test.append(label)
                    x_test.append(img)
                else:
                    y_train.append(label)
                    x_train.append(img)

    print('dataset size:', counter, '(train=%d, test=%d)' % (len(y_train), len(y_test)))
    return np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)

In [14]:
x_train, y_train, x_test, y_test = load_data(DATA_PATH)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

loading dataset...
dataset size: 7243 (train=6583, test=660)


In [15]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(6583, 25, 75, 1)
(6583, 2)
(660, 25, 75, 1)
(660, 2)


In [12]:
save_dir = os.path.join(PATH, 'saved_models')
model_name = 'keras_cifar10_trained_captcha_eval_model.h5'

In [16]:
input_layer = Input((int(IMG_ROW/2), int(IMG_COLS/2), 1))
x = Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(filters=48, kernel_size=(5, 5), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(filters=64, kernel_size=(5, 5), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Dropout(0.3)(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)

out = Dense(2, activation='softmax')(x)
model = Model(inputs=input_layer, outputs=out)

In [17]:
# initiate Adam optimizer
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


model.summary() 

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25, 75, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 25, 75, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 37, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 37, 48)        38448     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 18, 48)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 18, 64)         76864     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 9, 64)          0     

In [22]:
history = model.fit(x_train, y_train, batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1,
                    validation_data=(x_test, y_test)
                   )

Epoch 1/50
52/52 [==============================] - 14s 31ms/step - loss: 0.4533 - accuracy: 0.7689 - val_loss: 0.1565 - val_accuracy: 0.9455
Epoch 2/50
52/52 [==============================] - 1s 18ms/step - loss: 0.1113 - accuracy: 0.9612 - val_loss: 0.0890 - val_accuracy: 0.9773
Epoch 3/50
52/52 [==============================] - 1s 19ms/step - loss: 0.0930 - accuracy: 0.9703 - val_loss: 0.0735 - val_accuracy: 0.9833
Epoch 4/50
52/52 [==============================] - 1s 19ms/step - loss: 0.0617 - accuracy: 0.9829 - val_loss: 0.0724 - val_accuracy: 0.9833
Epoch 5/50
52/52 [==============================] - 1s 19ms/step - loss: 0.0405 - accuracy: 0.9877 - val_loss: 0.0507 - val_accuracy: 0.9833
Epoch 6/50
52/52 [==============================] - 1s 18ms/step - loss: 0.0349 - accuracy: 0.9889 - val_loss: 0.0889 - val_accuracy: 0.9727
Epoch 7/50
52/52 [==============================] - 1s 18ms/step - loss: 0.0399 - accuracy: 0.9865 - val_loss: 0.0616 - val_accuracy: 0.9803
Epoch 8/50
5

In [23]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at C:\Users\Qwerty\Downloads\captcha_solver\saved_models\keras_cifar10_trained_captcha_eval_model.h5 


In [25]:
# Score trained model.
scores = model.evaluate(x_train, y_train, verbose=1)
print('Train loss:     %f' % np.mean(scores[0]))
acc = 1.
acc *= scores[1]
print('Train accuracy: %.2f' % (acc * 100.))

206/206 [==============================] - 1s 4ms/step - loss: 3.3466e-06 - accuracy: 1.0000
Train loss:     0.000003
Train accuracy: 100.00
